In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from env import *
from time import sleep
import prepare

pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.reset_option("display.max_rows", "display.max_columns")

import re

In [2]:
# names = pd.read_csv('more_players.csv')

In [3]:
# names = names [['players']]

In [4]:
# names

In [5]:
# names[names.index==85]

In [6]:
# names

In [7]:
def get_puuid(name_list):
    puuid_list = []
    counter = 0
    for name in name_list:
        print(counter)
        counter += 1
        if counter%50 == 0:
            print('function paused')
            print
            time.sleep(180)
            print('Continue')
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            puuid_list.append(summoner_info)
        else:
            response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key={api_key}")
            summoner_info = response.json()
            puuid_list.append(summoner_info)
    return puuid_list

In [8]:
# puuid_list = get_puuid(names.players)

In [9]:
# puuid_list

In [10]:
# puuid_df = pd.DataFrame(puuid_list)

In [11]:
# puuid_df.drop(columns = 'status', inplace=True)

In [12]:
# puuid_df.dropna(inplace=True)

In [13]:
# puuid_df.to_csv('puuid.csv',index=False)

In [14]:
# puuid = puuid_df[['puuid']]

In [15]:
#match_ids = pd.read_csv('match_ids.csv')

In [16]:
#match_ids = list(match_ids['0'])[1000:2000]

In [17]:
# df = get_match_info(match_ids, api_key, username, time=20)

In [18]:
#other_game_data = pd.read_json('other_game_data_bryant.json')

In [19]:
#timeline_data = pd.read_json('timeline_data_bryant.json')

In [20]:
#timeline_list = timeline_data.to_dict(orient = 'records')

In [21]:
#other_game_list = other_game_data.to_dict(orient = 'records')

In [22]:
# df = prepare(timeline_list, other_game_list, time=20)

In [23]:
# df.head()

In [24]:
# df.size

In [25]:
# df.shape

In [26]:
# df.columns

In [27]:
# df.to_csv('bryant_games.csv')

In [28]:
# df.winningTeam.value_counts()

In [29]:
#Load the match_ids.csv file
matches_df = pd.read_csv('match_ids.csv')
matches_df

,0
0,NA1_4062082172
1,NA1_4093908534
2,NA1_4094915650
3,NA1_4082128299
4,NA1_4083500908
...,...
37279,NA1_4097058147
37280,NA1_4096721229
37281,NA1_4094049450
37282,NA1_4091691873


In [30]:
#Set up the match id list. You only need to uncomment the line following your name

match_ids = list(matches_df['0'])[10_657:17_314]

In [31]:
#Set up your username. You only need to uncomment the line following your name.
#Again, this username is only for naming the json files.

#Joshua Bryant
username = 'bryant'

In [32]:
def get_match_info(match_ids, api_key, username, time = 20):
    """
    This function takes in a list of match ids and iterates through them. For each iteration,
    it will make two api calls and retrieve the necessary information for our project.
    
    Two json lists will be created, and once all the info has been gathered for each match id,
    this function will save both json lists for future reference. 
    
    This function also takes in a username string. This will be used for naming the saved files.
    
    Finally, this function will funnel the resulting json lists into a prepare function and
    return a complete df.
    
    The time parameter is used in the prepare function that will be called at the end
    and represents the timeframe of the data we want to gather and prepare.
    The default value is 20 for the 20 minute mark of the match.
    
    """
    
    #Create an empty list to store the timeline json data
    timeline_data = []
    
    #Create an empty list to store the other json data
    game_data = []
    
    #Set up main url
    url = 'https://americas.api.riotgames.com/'
    
    #Create a count var
    n = 0
    
    #Store the length of the match_ids list in a var
    match_count = len(match_ids)
    
    #Loop through each match_id
    for match in match_ids:
        #Update the count var
        n += 1
        
        #After every 50 match ids (100 total requests), wait 150 seconds (2.5 minutes)
        if n % 50 == 0:
            print(f'Completed {n} of {match_count} match IDs.')
            print(f'Waiting 150 seconds...\n')
            
            #Sleep 150 seconds
            sleep(150)
            
            print('Continuing!\n')
        
        ############################# TIMELINE DATA #######################
        #Set up timeline url
        timeline_query = f'lol/match/v5/matches/{match}/timeline/?api_key={api_key}'
        
        #Grab timeline json data
        timeline_response = requests.get(url + timeline_query)
        
        #Check response and leave a status message
        if timeline_response.status_code != 200:
            print(f'Something went wrong getting TIMELINE DATA! Status Code {timeline_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID.\n')
            continue

        #Turn it into json format
        timeline_json = timeline_response.json()
        
        #Append this data to the timeline_data list
        timeline_data.append(timeline_json)
        
        ############################# OTHER GAME DATA #######################
        #Set up game data url
        game_query = f'lol/match/v5/matches/{match}?api_key={api_key}'
        
        #Grab game json data
        game_response = requests.get(url + game_query)
        
        #Check response and leave a status message.
        if game_response.status_code != 200:
            print(f'Something went wrong getting OTHER GAME DATA! Status Code {game_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.\n')
            
            #Remove the last entry in the timeline_data list
            timeline_data.pop()
            continue
        
        #Turn it into json format
        game_json = game_response.json()
        
        #Append this data to the game_data list
        game_data.append(game_json)
        
    ################################## END LOOP ##############################
    
    #Now that all of the json data has been gathered, save each file for future reference
    
    #Save timeline data.
    #First, convert it to a df
    timeline_df = pd.DataFrame(timeline_data)
    
    #Save as csv file. Use username string to identify whose file it is
    timeline_df.to_json(f'timeline_data_{username}.json')
    
    #Print status message
    print(f'Created timeline_data_{username}.json')
    
    #Save other game data
    #First, convert it to a df
    game_df = pd.DataFrame(game_data)
    
    #Save as csv file. Use username string to identify whose file it is
    game_df.to_json(f'other_game_data_{username}.json')
    
    #Print status message
    print(f'Created other_game_data_{username}.json\n')
    
    ################################ PREPARE DATA #########################
    
    #The following section will funnel the data into the prepare function
    #Written by Joshua C.
    
    #Leave a status message
    print('Preparing the data...\n')
    
    #Begin preparing the data
    df = prepare.prepare(timeline_data, game_data, time)
    
    #Finally, return the prepared df
    return df

In [33]:
#Before calling the get_match_info function, separate the match_ids list into smaller batches
match_batch_1 = match_ids[:1000]
match_batch_2 = match_ids[1000: 2000]
match_batch_3 = match_ids[2000: 3000]
match_batch_4 = match_ids[3000: 4000]
match_batch_5 = match_ids[4000: 5000]
match_batch_6 = match_ids[5000: 6000]
match_batch_7 = match_ids[6000: 6657]

In [34]:
#Since I will be running this function in smaller batches, I will have to update the username
#after each iteration. This will allow me to keep track of which json files contain what data

#For first iteration
#username = 'start_10_657_end_11_657'

#Call the get_match_info function
#df1 = get_match_info(match_batch_1, api_key, username)

In [35]:
#jsonList = pd.read_json('timeline_data_start_10_657_end_11_657.json')
#jsonList2 = pd.read_json('other_game_data_start_10_657_end_11_657.json')
#t = 20

In [36]:
#jsonList = jsonList.to_dict(orient='records')
#jsonList2 = jsonList2.to_dict(orient='records')

In [37]:
#df = prepare.prepare(jsonList, jsonList2, t)

In [38]:
#df.to_csv('bryant_start_10_657_end_11_657.csv')

In [39]:
#For second iteration
#username = 'start_11_657_end_12_657'

#Call the get_match_info function
#df2 = get_match_info(match_batch_2, api_key, username)

In [40]:
#df2.to_csv('bryant_start_11_657_end_12_657')

In [41]:
#For third iteration
#username = 'start_12_657_end_13_657'

#Call the get_match_info function
#df3 = get_match_info(match_batch_3, api_key, username)

In [42]:
#df3.to_csv('bryant_start_12_657_end_13_657')

In [43]:
#For the fourth iteration
#username = 'start_13_657_end_14_657'

#Call the get_match_info function
#df4 = get_match_info(match_batch_4, api_key, username)

In [44]:
#df4.to_csv('bryant_start_13_657_end_14_657')

In [45]:
#For the fifth iteration
#username = 'start_14_657_end_15_657'

#Call the get_match_info function
#df5 = get_match_info(match_batch_5, api_key, username)

In [46]:
#df5.to_csv('bryant_start_14_657_end_15_657')

In [47]:
#For the sixth iteration
username = 'start_15_657_end_16_657'

#Call the get_match_info function
df6 = get_match_info(match_batch_6, api_key, username)

Completed 50 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 650 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 700 of 1000 match IDs.
Waiting 150 seconds...

Continuing!

Completed 750 of 1000

Finished with: 212 of 1000
Skipping: 213 due to <20 min or not classic
Finished with: 214 of 1000
Finished with: 215 of 1000
Skipping: 216 due to <20 min or not classic
Finished with: 217 of 1000
Finished with: 218 of 1000
Skipping: 219 due to <20 min or not classic
Finished with: 220 of 1000
Finished with: 221 of 1000
Skipping: 222 due to <20 min or not classic
Finished with: 223 of 1000
Finished with: 224 of 1000
Finished with: 225 of 1000
Finished with: 226 of 1000
Finished with: 227 of 1000
Finished with: 228 of 1000
Finished with: 229 of 1000
Finished with: 230 of 1000
Skipping: 231 due to <20 min or not classic
Finished with: 232 of 1000
Finished with: 233 of 1000
Skipping: 234 due to <20 min or not classic
Skipping: 235 due to <20 min or not classic
Finished with: 236 of 1000
Finished with: 237 of 1000
Finished with: 238 of 1000
Finished with: 239 of 1000
Skipping: 240 due to <20 min or not classic
Finished with: 241 of 1000
Skipping: 242 due to <20 min or not classic
Finished w

Finished with: 474 of 1000
Skipping: 475 due to <20 min or not classic
Skipping: 476 due to <20 min or not classic
Finished with: 477 of 1000
Finished with: 478 of 1000
Finished with: 479 of 1000
Finished with: 480 of 1000
Finished with: 481 of 1000
Finished with: 482 of 1000
Skipping: 483 due to <20 min or not classic
Skipping: 484 due to <20 min or not classic
Finished with: 485 of 1000
Finished with: 486 of 1000
Finished with: 487 of 1000
Finished with: 488 of 1000
Finished with: 489 of 1000
Finished with: 490 of 1000
Finished with: 491 of 1000
Finished with: 492 of 1000
Finished with: 493 of 1000
Finished with: 494 of 1000
Finished with: 495 of 1000
Finished with: 496 of 1000
Finished with: 497 of 1000
Finished with: 498 of 1000
Finished with: 499 of 1000
Skipping: 500 due to <20 min or not classic
Finished with: 501 of 1000
Finished with: 502 of 1000
Finished with: 503 of 1000
Skipping: 504 due to <20 min or not classic
Skipping: 505 due to <20 min or not classic
Finished with: 50

Finished with: 732 of 1000
Finished with: 733 of 1000
Finished with: 734 of 1000
Finished with: 735 of 1000
Finished with: 736 of 1000
Finished with: 737 of 1000
Skipping: 738 due to <20 min or not classic
Finished with: 739 of 1000
Finished with: 740 of 1000
Finished with: 741 of 1000
Skipping: 742 due to <20 min or not classic
Skipping: 743 due to <20 min or not classic
Finished with: 744 of 1000
Finished with: 745 of 1000
Skipping: 746 due to <20 min or not classic
Skipping: 747 due to <20 min or not classic
Finished with: 748 of 1000
Finished with: 749 of 1000
Finished with: 750 of 1000
Finished with: 751 of 1000
Skipping: 752 due to <20 min or not classic
Finished with: 753 of 1000
Finished with: 754 of 1000
Finished with: 755 of 1000
Skipping: 756 due to <20 min or not classic
Skipping: 757 due to <20 min or not classic
Skipping: 758 due to <20 min or not classic
Finished with: 759 of 1000
Finished with: 760 of 1000
Finished with: 761 of 1000
Finished with: 762 of 1000
Finished w

Finished with: 991 of 1000
Skipping: 992 due to <20 min or not classic
Skipping: 993 due to <20 min or not classic
Finished with: 994 of 1000
Skipping: 995 due to <20 min or not classic
Finished with: 996 of 1000
Skipping: 997 due to <20 min or not classic
Finished with: 998 of 1000
Finished with: 999 of 1000


In [48]:
df6.to_csv('bryant_start_15_657_end_16_657')

In [49]:
#For the 7th iteration
username = 'start_16_657_end_17_314'

#Call the get_match_info function
df7 = get_match_info(match_batch_7, api_key, username)

Completed 50 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 650 of 657 match IDs.
Waiting 150 seconds...

Continuing!

Created timeline_data_start_16_657_end_17_314.json
Created other_game_data_start_16_657_end_17_314.json


Finished with: 240 of 657
Finished with: 241 of 657
Finished with: 242 of 657
Finished with: 243 of 657
Finished with: 244 of 657
Finished with: 245 of 657
Finished with: 246 of 657
Finished with: 247 of 657
Skipping: 248 due to <20 min or not classic
Finished with: 249 of 657
Finished with: 250 of 657
Skipping: 251 due to <20 min or not classic
Finished with: 252 of 657
Finished with: 253 of 657
Finished with: 254 of 657
Finished with: 255 of 657
Skipping: 256 due to <20 min or not classic
Skipping: 257 due to <20 min or not classic
Skipping: 258 due to <20 min or not classic
Skipping: 259 due to <20 min or not classic
Skipping: 260 due to <20 min or not classic
Finished with: 261 of 657
Skipping: 262 due to <20 min or not classic
Finished with: 263 of 657
Finished with: 264 of 657
Finished with: 265 of 657
Finished with: 266 of 657
Skipping: 267 due to <20 min or not classic
Skipping: 268 due to <20 min or not classic
Finished with: 269 of 657
Finished with: 270 of 657
Finished with:

Finished with: 503 of 657
Finished with: 504 of 657
Finished with: 505 of 657
Finished with: 506 of 657
Skipping: 507 due to <20 min or not classic
Finished with: 508 of 657
Finished with: 509 of 657
Skipping: 510 due to <20 min or not classic
Finished with: 511 of 657
Finished with: 512 of 657
Skipping: 513 due to <20 min or not classic
Finished with: 514 of 657
Finished with: 515 of 657
Finished with: 516 of 657
Skipping: 517 due to <20 min or not classic
Skipping: 518 due to <20 min or not classic
Finished with: 519 of 657
Finished with: 520 of 657
Finished with: 521 of 657
Finished with: 522 of 657
Finished with: 523 of 657
Skipping: 524 due to <20 min or not classic
Finished with: 525 of 657
Finished with: 526 of 657
Skipping: 527 due to <20 min or not classic
Finished with: 528 of 657
Finished with: 529 of 657
Finished with: 530 of 657
Finished with: 531 of 657
Finished with: 532 of 657
Finished with: 533 of 657
Finished with: 534 of 657
Skipping: 535 due to <20 min or not classi

In [50]:
df7.to_csv('bryant_start_16_657_end_17_314')

In [53]:
df = pd.read_csv('bryant_start_10_657_end_11_657.csv')
df2 = pd.read_csv('bryant_start_11_657_end_12_657.csv')
df3 = pd.read_csv('bryant_start_12_657_end_13_657.csv')
df4 = pd.read_csv('bryant_start_13_657_end_14_657.csv')
df5 = pd.read_csv('bryant_start_14_657_end_15_657.csv')
df6 = pd.read_csv('bryant_start_15_657_end_16_657.csv')

In [54]:
frames = [df,df2,df3,df4,df5,df6,df7]

In [55]:
result = pd.concat(frames)

In [56]:
result.shape

(4765, 209)

In [57]:
result.to_csv('bryant_10_657_end_16_657.csv')